In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#print(os.listdir("../input"))


In [15]:
# File loading
df  = pd.read_csv('./unigram_freq.csv/unigram_freq.csv')
print(df.shape)
df.dropna(axis=0,how='any')
print(df.shape)


(333333, 2)
(333333, 2)


In [16]:
lines = [x for x in df['word'] if type(x) == type('a') ]
print("Line Count:",len(lines))
print(lines[:4])

Line Count: 333331
['the', 'of', 'and', 'to']


In [17]:
# Preprocessing
import re
def process(sent):
    sent=sent.lower()
    sent=re.sub(r'[^0-9a-zA-Z ]','',sent)
    sent=sent.replace('\n','')
    return sent    

In [18]:
lines =[process(x) for x in lines]
temp = []
for line in lines:
    temp+= [ x for x in line.split() ]
lines = list(set(temp))
print("\n".join(lines[:4]))
print("Number of items:",len(lines))

lauritzen
crevice
libcap
rocas
Number of items: 333331


In [19]:
# CHAR INDEXING
char_set = list(" abcdefghijklmnopqrstuvwxyz0123456789")
char2int = { char_set[x]:x for x in range(len(char_set)) }
int2char = { char2int[x]:x for x in char_set }
print(char2int)
print(int2char)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36}
{0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '0', 28: '1', 29: '2', 30: '3', 31: '4', 32: '5', 33: '6', 34: '7', 35: '8', 36: '9'}


In [20]:
count = len(char_set)
codes = ["\t","\n",'#']
for i in range(len(codes)):
    code = codes[i]
    char2int[code]=count
    int2char[count]=code
    count+=1
print(char2int)
print(int2char)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, '\t': 37, '\n': 38, '#': 39}
{0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '0', 28: '1', 29: '2', 30: '3', 31: '4', 32: '5', 33: '6', 34: '7', 35: '8', 36: '9', 37: '\t', 38: '\n', 39: '#'}


In [21]:
import random
#thresh - 0 to 1
def gen_gibberish(line,thresh=0.2):
    times = int(random.randrange(1,len(line)) * thresh)
    '''
    Types of replacement:
        1.Delete random character.
        2.Add random character.
        3.Replace a character.
        4.Combination?
    '''
    while times!=0:
        # try to gen noise length times...
        times-=1
        val = random.randrange(0,10)
        if val <= 5:
            #get random index
            val = random.randrange(0,10)
            index = random.randrange(2,len(line))
            if val <= 3 :
                #delete character
                line = line[:index]+line[index+1:]
            else:
                #add character
                insert_index = random.randrange(0,len(char_set))
                line = line[:index] + char_set[insert_index] + line[index:]
        else:
            index = random.randrange(0,len(char_set))
            replace_index = random.randrange(2,len(line))
            line = line[:replace_index] + char_set[index] + line[replace_index+1:]
    return line

sample = lines[5]
gib = gen_gibberish(sample)
print("Original:",sample)
print("Gibberish:",gib)
        
    

Original: ficedula
Gibberish: ficedyula


In [22]:
# create dataset
input_texts = []
target_texts = []
REPEAT_FACTOR = 1
SKIP = int(len(lines)*0.65)

for line in lines[SKIP:]:
    if len(line)>10:
        output_text = '\t' + line + '\n'
        for _ in range(REPEAT_FACTOR):
            input_text = gen_gibberish(line)
            input_texts.append(input_text)
            target_texts.append(output_text)
print("LEN OF SAMPLES:",len(input_texts))

LEN OF SAMPLES: 15303


In [23]:
max_enc_len = max([len(x) for x in input_texts])
max_dec_len = max([len(x) for x in target_texts])
print("Max Enc Len:",max_enc_len)
print("Max Dec Len:",max_dec_len)

Max Enc Len: 39
Max Dec Len: 38


In [24]:
num_samples = len(input_texts)
encoder_input_data = np.zeros( (num_samples , max_enc_len , len(char_set)),dtype='float32' )
decoder_input_data = np.zeros( (num_samples , max_dec_len , len(char_set)+2),dtype='float32' )
decoder_target_data = np.zeros( (num_samples , max_dec_len , len(char_set)+2),dtype='float32' )
print("CREATED ZERO VECTORS")

CREATED ZERO VECTORS


In [25]:
#filling in the enc,dec datas
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[ i , t , char2int[char] ] = 1
    for t,char in enumerate(target_text):
        decoder_input_data[ i, t , char2int[char] ] = 1
        if t > 0 :
            decoder_target_data[ i , t-1 , char2int[char] ] = 1
print("COMPLETED...")    

COMPLETED...


In [26]:
from keras.models import Model
from keras.layers import Input,LSTM,Dense

In [27]:
batch_size = 128
epochs = 250
latent_dim = 256

num_enc_tokens = len(char_set)
num_dec_tokens = len(char_set) + 2 # includes \n \t
encoder_inputs = Input(shape=(None,num_enc_tokens))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs , state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h,state_c]


In [28]:
decoder_inputs = Input(shape=(None,num_dec_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_ouputs,_,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)

decoder_dense = Dense(num_dec_tokens, activation='softmax')
decoder_ouputs = decoder_dense(decoder_ouputs)

model = Model([encoder_inputs,decoder_inputs],decoder_ouputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 37)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 39)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        301056      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
h=model.fit([encoder_input_data,decoder_input_data],decoder_target_data
         ,epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.2
         )
model.save('s2s.h5')


Epoch 1/250
96/96 [==============================] - 63s 562ms/step - loss: 1.1013 - val_loss: 1.0544
Epoch 2/250
96/96 [==============================] - 51s 534ms/step - loss: 1.0554 - val_loss: 1.0359
Epoch 3/250
96/96 [==============================] - 51s 527ms/step - loss: 1.0489 - val_loss: 1.0399
Epoch 4/250
96/96 [==============================] - 51s 529ms/step - loss: 1.0429 - val_loss: 1.0287
Epoch 5/250
96/96 [==============================] - 52s 540ms/step - loss: 1.0368 - val_loss: 1.0356
Epoch 6/250
96/96 [==============================] - 51s 528ms/step - loss: 1.0306 - val_loss: 1.0202
Epoch 7/250
96/96 [==============================] - 51s 534ms/step - loss: 1.0234 - val_loss: 1.0121
Epoch 8/250
96/96 [==============================] - 52s 540ms/step - loss: 1.0158 - val_loss: 1.0090
Epoch 9/250
96/96 [==============================] - 52s 537ms/step - loss: 1.0102 - val_loss: 1.0000
Epoch 10/250
96/96 [==============================] - 51s 534ms/step - loss: 1.004

96/96 [==============================] - 51s 527ms/step - loss: 0.8743 - val_loss: 0.8718
Epoch 81/250
96/96 [==============================] - 53s 552ms/step - loss: 0.8690 - val_loss: 0.8774
Epoch 82/250
96/96 [==============================] - 53s 548ms/step - loss: 0.8659 - val_loss: 0.8653
Epoch 83/250
96/96 [==============================] - 51s 535ms/step - loss: 0.8649 - val_loss: 0.8688
Epoch 84/250
96/96 [==============================] - 50s 526ms/step - loss: 0.8593 - val_loss: 0.8634
Epoch 85/250
96/96 [==============================] - 51s 531ms/step - loss: 0.8568 - val_loss: 0.8638
Epoch 86/250
96/96 [==============================] - 49s 514ms/step - loss: 0.8548 - val_loss: 0.8556
Epoch 87/250
96/96 [==============================] - 50s 524ms/step - loss: 0.8553 - val_loss: 0.8554
Epoch 88/250
96/96 [==============================] - 49s 514ms/step - loss: 0.8518 - val_loss: 0.8571
Epoch 89/250
96/96 [==============================] - 49s 516ms/step - loss: 0.8511 - 

96/96 [==============================] - 35s 362ms/step - loss: 0.7110 - val_loss: 0.7238
Epoch 160/250
96/96 [==============================] - 35s 366ms/step - loss: 0.7059 - val_loss: 0.7239
Epoch 161/250
96/96 [==============================] - 36s 375ms/step - loss: 0.7037 - val_loss: 0.7222
Epoch 162/250
96/96 [==============================] - 35s 367ms/step - loss: 0.7064 - val_loss: 0.7206
Epoch 163/250
96/96 [==============================] - 36s 370ms/step - loss: 0.7001 - val_loss: 0.7203
Epoch 164/250
96/96 [==============================] - 36s 372ms/step - loss: 0.6991 - val_loss: 0.7179
Epoch 165/250
96/96 [==============================] - 35s 370ms/step - loss: 0.6950 - val_loss: 0.7143
Epoch 166/250
96/96 [==============================] - 35s 368ms/step - loss: 0.6890 - val_loss: 0.7095
Epoch 167/250
96/96 [==============================] - 35s 369ms/step - loss: 0.6897 - val_loss: 0.7076
Epoch 168/250
96/96 [==============================] - 36s 372ms/step - loss: 

In [ ]:
plt.plot(h.history['loss'])
plt.title('Model Loss')
plt.show()

In [ ]:
encoder_model = Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c = decoder_lstm(
        decoder_inputs,initial_state = decoder_states_inputs
)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)
encoder_model.save('encoder.h5')
decoder_model.save('decoder.h5')

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_dec_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, char2int['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int2char[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_dec_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

for seq_index in range(700):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Wrong sentence:', input_texts[seq_index])
    print('Corrected sentence:', decoded_sentence)
    print('Ground Truth:',target_texts[seq_index])